In [1]:
import numpy as np
import pandas as pd
import xgboost

df=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')

In [2]:
df.Response.value_counts()

0    38673
1    12209
Name: Response, dtype: int64

In [3]:
X=df.iloc[:,:-1]
X_Sub=df_test.iloc[:,:]
y=df.iloc[:,-1]

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=27)

In [5]:
df_X = pd.concat([X_train, y_train], axis=1)

In [6]:
df_X.Response.value_counts()

0    30985
1     9720
Name: Response, dtype: int64

In [7]:
df_majority = df_X[df_X.Response==0]
df_minority = df_X[df_X.Response==1]

In [8]:
df_majority.Response.value_counts()

0    30985
Name: Response, dtype: int64

In [9]:
df_minority.Response.value_counts()

1    9720
Name: Response, dtype: int64

In [10]:
from sklearn.utils import resample
df_majority_downsampled = resample(df_majority,replace=False,n_samples=len(df_minority),random_state=1234)

In [11]:
df_majority_downsampled.Response.value_counts()

0    9720
Name: Response, dtype: int64

In [12]:
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

In [13]:
df_downsampled.Response.value_counts()

1    9720
0    9720
Name: Response, dtype: int64

In [14]:
df_downsampled.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
15848,15849,C7,1473,Owned,Individual,34,34,No,X4,NaN,NaN,19,13260.0,0
24279,24280,C4,1365,Owned,Joint,67,66,Yes,X1,NaN,NaN,18,26976.0,0
23436,23437,C11,53,Owned,Individual,34,34,No,X3,2.0,1.0,22,12276.0,0
28338,28339,C3,372,Owned,Individual,29,29,No,X2,NaN,NaN,21,11322.0,0
19942,19943,C10,45,Owned,Individual,27,27,No,X1,NaN,NaN,16,7404.0,0


In [15]:
df_downsampled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19440 entries, 15848 to 4848
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       19440 non-null  int64  
 1   City_Code                19440 non-null  object 
 2   Region_Code              19440 non-null  int64  
 3   Accomodation_Type        19440 non-null  object 
 4   Reco_Insurance_Type      19440 non-null  object 
 5   Upper_Age                19440 non-null  int64  
 6   Lower_Age                19440 non-null  int64  
 7   Is_Spouse                19440 non-null  object 
 8   Health Indicator         15031 non-null  object 
 9   Holding_Policy_Duration  11770 non-null  object 
 10  Holding_Policy_Type      11770 non-null  float64
 11  Reco_Policy_Cat          19440 non-null  int64  
 12  Reco_Policy_Premium      19440 non-null  float64
 13  Response                 19440 non-null  int64  
dtypes: float64(2), int6

In [16]:
X_train=df_downsampled.iloc[:,:-1]
y_train=df_downsampled.iloc[:,-1]

In [21]:
X_train=X_train.loc[:,['Reco_Policy_Premium','Region_Code','Reco_Policy_Cat','Upper_Age','Holding_Policy_Duration','Reco_Insurance_Type']]
X_test=X_test.loc[:,['Reco_Policy_Premium','Region_Code','Reco_Policy_Cat','Upper_Age','Holding_Policy_Duration','Reco_Insurance_Type']]
X_Sub=X_Sub.loc[:,['Reco_Policy_Premium','Region_Code','Reco_Policy_Cat','Upper_Age','Holding_Policy_Duration','Reco_Insurance_Type']]

In [29]:
cat=['Reco_Insurance_Type']
num=[i for i in X_train.columns if i not in cat]

In [30]:
X1_train=X_train.loc[:,cat]
X2_train=X_train.loc[:,num]


In [31]:
X1_test=X_test.loc[:,cat]
X2_test=X_test.loc[:,num]

X1_Sub=X_Sub.loc[:,cat]
X2_Sub=X_Sub.loc[:,num]

In [34]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X1_train=X1_train.apply(le.fit_transform)

In [35]:
X1_test=X1_test.apply(LabelEncoder().fit_transform)
X1_Sub=X1_Sub.apply(LabelEncoder().fit_transform)

In [36]:
unique_items2=X2_train['Holding_Policy_Duration'].unique()
map_dict2={}
for i,j in enumerate(unique_items2):
    map_dict2[j]=i
map_dict2[np.nan]=np.nan
X2_train['Holding_Policy_Duration']=X2_train['Holding_Policy_Duration'].map(map_dict2)

In [37]:
X2_test['Holding_Policy_Duration']=X2_test['Holding_Policy_Duration'].map(map_dict2)
X2_Sub['Holding_Policy_Duration']=X2_Sub['Holding_Policy_Duration'].map(map_dict2)

In [39]:
X_train=pd.concat([X1_train,X2_train],axis=1)
X_test=pd.concat([X1_test,X2_test],axis=1)
X_Sub=pd.concat([X1_Sub,X2_Sub],axis=1)

In [40]:
clf = xgboost.XGBClassifier()
clf.fit(X_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:40:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [41]:
y_pred=clf.predict(X_test)

In [43]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.5766478122887426

In [44]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy_score(y_test, y_pred)

0.5533064753856736

In [45]:
confusion_matrix(y_test, y_pred)

array([[4082, 3606],
       [ 940, 1549]], dtype=int64)

In [46]:
Y_Sub=clf.predict(X_Sub)

In [47]:
id=pd.DataFrame(df_test.ID,columns=['ID'])
respo=pd.DataFrame(Y_Sub,columns=['Response'])
pred=pd.concat([id,respo],axis=1)

In [48]:
pred.head()

,ID,Response
0,50883,1
1,50884,0
2,50885,0
3,50886,0
4,50887,1


In [49]:
filename='Submission9.csv'
pred.to_csv(filename,index=False)